In [1]:
import tensorflow as tf # a default graph is created with a tf import
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# the contents of these variables are the *outputs* of the operations
# and not operations themselves

a = tf.constant(5, name = "my_a") # output of a const value
b = tf.constant(2, name = "my_b")
c = tf.constant(3, name = "my_c")

d = tf.multiply(a, b, name = "my_d")
e = tf.add(c, b, name = "my_e")
f = tf.subtract(d, e, name = "my_f")
g = tf.logical_xor(True, False)

### Fetches

In [3]:
# the graph computes only essential nodes according to the 
# dependencies (here - of "outs")
with tf.Session() as sess:
    fetches = [a,b,c,d,e,f]  # fetches hold the element of the graph
                             # we want to compute
    outs = sess.run(fetches) # execution
    
print(outs)
print(type(outs[0]))         # list of numpy elements

[5, 2, 3, 10, 5, 5]
<type 'numpy.int32'>


In [4]:
# we can perform the same runs in the same graph multiple times
# but each in a new session
with tf.Session() as sess:
    out = sess.run(f)
print(type(f))
print(f)
print(type(out))
print(out)

<class 'tensorflow.python.framework.ops.Tensor'>
Tensor("my_f:0", shape=(), dtype=int32)
<type 'numpy.int32'>
5


### Expanding dims

In [5]:
A = tf.constant([[1,2,3], [4,5,6]])
x = tf.constant([7, 8, 9])
print(A.get_shape()) 
print(x.get_shape())
# need to reshape x
x = tf.expand_dims(x, 1)
print(x.get_shape())
b = tf.matmul(A, x)

sess = tf.InteractiveSession()
print(b.eval())
sess.close()

(2, 3)
(3,)
(3, 1)
[[ 50]
 [122]]


### Graphs

In [6]:
# compare the default graph and another one
print(tf.get_default_graph())
g = tf.Graph()
print(g)

# check which graph is associated with a node
s = tf.constant(10)
print(s.graph)

# use contect manager (with statement) to run with a given graph
with g.as_default():
    print(g is tf.get_default_graph())

True


In [7]:
with tf.Graph().as_default():
    print(tf.constant(1, name = "my")) # names should be unique *in the same graph*;
    print(tf.constant(2, name = "my")) # tf adds "_<n>" otherwise

Tensor("my:0", shape=(), dtype=int32)
Tensor("my_1:0", shape=(), dtype=int32)


### Name scopes

In [8]:
with tf.Graph().as_default():
    with tf.name_scope("prefix_A"):
        print(tf.constant(1, name = "p")) 
        print(tf.constant(2, name = "q"))
        with tf.name_scope("prefix_B"):
            print(tf.constant(11, name = "r"))
            print(tf.constant(22, name = "s"))

Tensor("prefix_A/p:0", shape=(), dtype=int32)
Tensor("prefix_A/q:0", shape=(), dtype=int32)
Tensor("prefix_A/prefix_B/r:0", shape=(), dtype=int32)
Tensor("prefix_A/prefix_B/s:0", shape=(), dtype=int32)


### Variables

Variables are a special type of Tensors that can maintain a "fixed" state in a graph.
Using them requires they are initialized.

In [9]:
init_val = tf.random_normal((1, 4), 0 , 1)
var = tf.Variable(init_val)
print(var)

try:
    with tf.Session() as sess:
        result = sess.run(var)
except tf.errors.FailedPreconditionError:
    print "It'll throw with a mesg Attempting to use uninitialized value Variable_6"

# Now initialize the Variable
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    result = sess.run(var)
    print(result)
    
# with each run a new Variable is created - the name will have "_n" appended to name: 'Variable_11:0'

<tf.Variable 'Variable:0' shape=(1, 4) dtype=float32_ref>
It'll throw with a mesg Attempting to use uninitialized value Variable_6
[[-0.2057834   0.60403806 -1.5744915  -0.70292491]]


### Placeholders

In [10]:
# if shape is None or empty, anu size is accepted
pa = tf.placeholder(tf.float32, shape=(None, 10))
print(pa)

Tensor("Placeholder:0", shape=(?, 10), dtype=float32)


In [11]:
_x = np.random.randn(5, 10)
_y = np.random.randn(10, 1)

with tf.Graph().as_default():
    x = tf.placeholder(tf.float32, shape=(5, 10))
    y = tf.placeholder(tf.float32, shape=(10, 1))
    xy = tf.matmul(x, y)
    argmax = tf.argmax(xy)

    with tf.Session() as sess:
        matr = sess.run(xy, feed_dict={x: _x, y: _y})
        # scalar = sess.run(argmax, feed_dict={x: _x, y: _y}) # hm, why does this work??
        scalar = sess.run(argmax, feed_dict={xy: matr})

print(matr)
print(scalar)

[[-2.22163916]
 [-4.7881918 ]
 [-3.03102112]
 [ 1.55688775]
 [ 0.86975569]]
[3]


### Optimization

#### Multivariate linear regression

In [12]:
x = tf.placeholder(tf.float32, shape=(None, 3))
y_true = tf.placeholder(tf.float32, shape=(None, 3))
w = tf.Variable([[0,0,0]], dtype=tf.float32, name = 'weights')
b = tf.Variable(0, dtype=tf.float32, name = 'intercept')

y_pred = tf.matmul(tf.transpose(w), x) + b

#### Loss function

In [13]:
# Mean Square Error
loss_mse = tf.reduce_mean(tf.square(y_true - y_pred))

# Cross entropy (esp. for categorical data)
loss_cross_entropy = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true, logits=y_pred))

#### Optimize the loss function

In [14]:
# In Tensorflow gradients are calculated using "automatic differentiation"
# learning rate - determines how large is the step in the direction of negative gradient
# (we don't want to overshoot the minimum)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(loss_mse)

### Logistic regression

In [15]:
def plot(vals):
    plt.plot(vals)
    plt.show()

In [16]:
def sigmoid(i):
    return 1.0 / (1 + np.exp(-i))

In [18]:
N = 20000
x_data = np.random.randn(N, 3)
w_real = [0.3, 0.5, 0.1] # chosen weights
b = -0.2                 # chosen bias
wb = np.matmul(w_real, x_data.T) + b # (w, x_data.T): (1,3)x(3,N) -> 1xN

y_data_pre_noise = sigmoid(wb)
y_data = np.random.binomial(1, y_data_pre_noise)

NUM_STEPS = 50
LEARNING_RATE = 0.5

g = tf.Graph()
wb_ = []
with g.as_default():
    x = tf.placeholder(tf.float32, shape=[None,3])
    y_true = tf.placeholder(tf.float32, shape=None)
    
    with tf.name_scope('inference') as scope:
        w = tf.Variable([[0,0,0]], dtype=tf.float32, name = 'weights')
        b = tf.Variable(0, dtype=tf.float32, name = 'bias')
        y_pred = tf.matmul(w, tf.transpose(x)) + b

    with tf.name_scope('loss') as scope:
        loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true,logits=y_pred) 
        loss = tf.reduce_mean(loss)
  
    with tf.name_scope('train') as scope:
        optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE)
        train = optimizer.minimize(loss)

    # Initialize the vars and run the training
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)      
        for step in range(NUM_STEPS):
            sess.run(train, {x: x_data, y_true: y_data})
            if (step % 5 == 0):
                print(step, sess.run([w,b]))
                wb_.append(sess.run([w,b]))

        print(50, sess.run([w,b]))
        
        # The weights and bias are approaching the chosen/real [0.3, 0.5, 0.1] and -0.2

(0, [array([[ 0.03611456,  0.05677616,  0.01069056]], dtype=float32), -0.021125071])
(5, [array([[ 0.15908694,  0.25078008,  0.04631922]], dtype=float32), -0.093046457])
(10, [array([[ 0.22413744,  0.35402197,  0.06443311]], dtype=float32), -0.130997])
(15, [array([[ 0.26012734,  0.41144741,  0.07407537]], dtype=float32), -0.15188837])
(20, [array([[ 0.28063321,  0.44432271,  0.0793708 ]], dtype=float32), -0.1637181])
(25, [array([[ 0.29252315,  0.46346644,  0.08233628]], dtype=float32), -0.17053276])
(30, [array([[ 0.29948896,  0.47472504,  0.08401754]], dtype=float32), -0.17449962])
(35, [array([[ 0.30359507,  0.48138458,  0.08497842]], dtype=float32), -0.17682363])
(40, [array([[ 0.30602437,  0.48533687,  0.08553064]], dtype=float32), -0.17819072])
(45, [array([[ 0.30746484,  0.48768702,  0.08584926]], dtype=float32), -0.17899701])
(50, [array([[ 0.30818272,  0.48886099,  0.08600436]], dtype=float32), -0.17939705])
